<a href="https://colab.research.google.com/github/stefymojica/sample-rag-llm/blob/main/samples-rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**EJEMPLO DE UN RAG CON SOLO DOCUMENTOS, ES DECIR, NO UTILIZA LLM PARA GENERAR TEXTOS UNIFICADOS SINO QUE SOLO RETORNA FRAGMENTOS DE TEXTO**

In [ ]:
pip install sentence-transformers numpy scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 4.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

class SimpleRAG:
    def __init__(self):
        # Inicializar el modelo de embeddings
        self.embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
        self.documents = []
        self.embeddings = []

    def add_documents(self, documents):
        """
        Añade documentos a la base de conocimiento y genera sus embeddings
        """
        self.documents.extend(documents)
        new_embeddings = self.embedding_model.encode(documents)
        if len(self.embeddings) == 0:
            self.embeddings = new_embeddings
        else:
            self.embeddings = np.vstack([self.embeddings, new_embeddings])

    def retrieve(self, query, top_k=3):
        """
        Recupera los documentos más relevantes para una consulta
        """
        # Generar embedding de la consulta
        query_embedding = self.embedding_model.encode([query])[0]

        # Calcular similitud con todos los documentos
        similarities = cosine_similarity([query_embedding], self.embeddings)[0]

        # Obtener los índices de los documentos más similares
        top_indices = np.argsort(similarities)[-top_k:][::-1]

        # Devolver los documentos más relevantes y sus puntuaciones
        results = []
        for idx in top_indices:
            results.append({
                'document': self.documents[idx],
                'score': similarities[idx]
            })

        return results

    def generate_response(self, query):
        """
        Genera una respuesta basada en los documentos recuperados
        """
        relevant_docs = self.retrieve(query)

        # Crear un contexto con los documentos relevantes
        context = "\n".join([f"Documento {i+1}: {doc['document']}"
                           for i, doc in enumerate(relevant_docs)])

        # En un sistema real, aquí se usaría un LLM para generar la respuesta
        # Por simplicidad, solo devolvemos el contexto
        return f"Basado en los siguientes documentos:\n{context}"

# Ejemplo de uso
def main():
    # Crear instancia del RAG
    rag = SimpleRAG()

    # Añadir algunos documentos de ejemplo
    documentos = [
        "Python es un lenguaje de programación interpretado y de alto nivel.",
        "JavaScript es un lenguaje de programación para desarrollo web.",
        "Los frameworks de Python incluyen Django y Flask.",
        "React es una biblioteca de JavaScript para construir interfaces.",
        "Python es muy popular en ciencia de datos y machine learning.",
        "javascript es de los lenguajes mas importantes de el mundo",
    ]

    rag.add_documents(documentos)

    # Realizar una consulta
    query = "¿Qué es Javascript?"
    response = rag.generate_response(query)
    print(f"\nConsulta: {query}")
    print("\nRespuesta:")
    print(response)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Consulta: ¿Qué es Javascript?

Respuesta:
Basado en los siguientes documentos:
Documento 1: JavaScript es un lenguaje de programación para desarrollo web.
Documento 2: javascript es de los lenguajes mas importantes de el mundo
Documento 3: React es una biblioteca de JavaScript para construir interfaces.


**EJEMPLO DE UN MODELO CON RAG Y LLM, EN DONDE EL MODELO SI TRAE UNA INFORMACION CON SENTIDO Y UNIFICADA**

In [5]:
pip install torch transformers sentence-transformers scikit-learn

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import torch
import logging
from typing import List, Dict
from datetime import datetime

class ImprovedRAG:
    def __init__(self):
        self.setup_logging()
        self.setup_models()
        self.documents = []
        self.embeddings = []

    def setup_logging(self):
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

    def setup_models(self):
        try:
            self.logger.info("Inicializando modelos...")

            # Modelo de embeddings
            self.embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

            # Modelo LLM
            self.llm = pipeline(
                "text-generation",
                model="facebook/opt-1.3b",
                device=0 if torch.cuda.is_available() else -1,
                model_kwargs={"torch_dtype": torch.float16}
            )

            self.logger.info("Modelos inicializados correctamente")
        except Exception as e:
            self.logger.error(f"Error inicializando modelos: {e}")
            raise

    def add_documents(self, documents: List[str]):
        try:
            self.logger.info(f"Añadiendo {len(documents)} documentos")

            new_embeddings = self.embedding_model.encode(documents)
            self.documents.extend(documents)

            if len(self.embeddings) == 0:
                self.embeddings = new_embeddings
            else:
                self.embeddings = np.vstack([self.embeddings, new_embeddings])

            self.logger.info("Documentos añadidos correctamente")
        except Exception as e:
            self.logger.error(f"Error añadiendo documentos: {e}")
            raise

    def retrieve(self, query: str, top_k: int = 3) -> List[Dict]:
        try:
            query_embedding = self.embedding_model.encode([query])[0]
            similarities = cosine_similarity([query_embedding], self.embeddings)[0]

            top_indices = np.argsort(similarities)[-top_k:][::-1]

            return [
                {
                    'document': self.documents[idx],
                    'score': float(similarities[idx])
                }
                for idx in top_indices
            ]
        except Exception as e:
            self.logger.error(f"Error en recuperación: {e}")
            raise

    def generate_response(self, query: str) -> Dict:
        try:
            start_time = datetime.now()

            relevant_docs = self.retrieve(query)
            context = "\n".join([doc['document'] for doc in relevant_docs])

            prompt = f"""
            Instrucciones: A partir del siguiente contexto, genera una respuesta completa
            y coherente a la pregunta. Sintetiza la información relevante y proporciona
            una explicación clara.

            Contexto:
            {context}

            Pregunta: {query}

            Respuesta:"""

            response = self.llm(
                prompt,
                max_length=300,
                num_return_sequences=1,
                temperature=0.7,
                top_p=0.95,
                do_sample=True,
                no_repeat_ngram_size=3,
                top_k=50,
                early_stopping=True
            )[0]['generated_text']

            # Extraer solo la respuesta generada
            response = response.split("Respuesta:")[-1].strip()

            processing_time = (datetime.now() - start_time).total_seconds()

            return {
                'query': query,
                'response': response,
                'retrieved_documents': relevant_docs,
                'processing_time': processing_time
            }
        except Exception as e:
            self.logger.error(f"Error generando respuesta: {e}")
            raise

    def batch_process(self, queries: List[str]) -> List[Dict]:
        """Procesar múltiples consultas"""
        return [self.generate_response(query) for query in queries]

def main():
    # Inicializar el sistema
    rag = ImprovedRAG()

    # Documentos de ejemplo
    documents = [
        "Python es un lenguaje de programación interpretado y de alto nivel.",
        "JavaScript es un lenguaje de programación para desarrollo web.",
        "Los frameworks de Python incluyen Django y Flask.",
        "React es una biblioteca de JavaScript para construir interfaces.",
        "Python es muy popular en ciencia de datos y machine learning.",
        "JavaScript se ejecuta en el navegador del cliente.",
        "Python tiene una sintaxis clara y legible.",
        "JavaScript permite crear aplicaciones web interactivas.",
        "Python cuenta con una gran comunidad de desarrolladores.",
        "JavaScript es fundamental para el desarrollo frontend."
    ]

    # Añadir documentos
    rag.add_documents(documents)

    # Consultas de prueba
    queries = [
        "¿Qué es Python y cuáles son sus características principales?",
    ]

    # Procesar consultas
    for query in queries:
        print(f"\nConsulta: {query}")
        result = rag.generate_response(query)

        print(f"\nRespuesta generada:")
        print(result['response'])

        print("\nDocumentos recuperados:")
        for doc in result['retrieved_documents']:
            print(f"- Score: {doc['score']:.4f}")
            print(f"  {doc['document']}")

        print(f"\nTiempo de proceso: {result['processing_time']:.2f} segundos")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

...

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Consulta: ¿Qué es Python y cuáles son sus características principales?


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(



Respuesta generada:
Python es el programa de programa.

Python es una plataforma de software para desarrolar cómo func

Documentos recuperados:
- Score: 0.7316
  Python es un lenguaje de programación interpretado y de alto nivel.
- Score: 0.6698
  Python cuenta con una gran comunidad de desarrolladores.
- Score: 0.6595
  Python tiene una sintaxis clara y legible.

Tiempo de proceso: 746.12 segundos

Consulta: ¿Para qué se utiliza JavaScript en el desarrollo web?


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
